# Copy files from merged into standard structure
TODO: Some of this code "fixes" filenaming that needs to be pushed back into Wintap

In [1]:
def parse_filename(filename):
    """
    Legacy format: hostname=event_type+epoch_ts.parquet
    New format:    hostname+event_type+epoch_ts.parquet
    """
    if "=" in filename:
        # Legacy
        hostname = filename.split("=")[0]
        event_type = filename.split("=")[1].rsplit("+")[0]
        data_capture_epoch = filename.split("=")[1].rsplit("+")[1].split(".")[0]
    else:
        hostname = filename.split("+")[0]
        event_type = filename.split("+")[1]
        # Drop the '.parquet' also
        data_capture_epoch = filename.split("+")[2].split(".")[0]
    return hostname, event_type, int(data_capture_epoch)


def win32_to_epoch(wts):
    return wts / 1e7 - 11644473600

In [2]:
# Merged directory is flat, so no need for recursive listing
import os
import shutil
from datetime import datetime, timedelta, timezone

# srcdir="c:/ProgramData/Wintap/parquet/merged"
srcdir = "c:/WintapData/wd3/merged"
local_path = "c:/WintapData/wd3"

for file in os.listdir(srcdir):
    if file.endswith(".parquet"):
        (hostname, event_type, data_capture_epoch) = parse_filename(file)
        # Copy to standard structure
        data_capture_ts = datetime.fromtimestamp(
            int(win32_to_epoch(data_capture_epoch)), timezone.utc
        )
        datadpk = data_capture_ts.strftime("%Y%m%d")
        datahpk = data_capture_ts.strftime("%H")
        # Define fully-qualified local name
        if any([x in event_type for x in ["tcp", "udp"]]):
            # Special handling of TCP/UDP
            proto = event_type[4:7]
            # Force the correct supertype
            event_type = "raw_process_conn_incr"
            local_file_path = f"{local_path}/raw_sensor/{event_type}/dayPK={datadpk}/hourPK={datahpk}/proto={proto}"
        else:
            match event_type:
                case "raw_file":
                    event_type = "raw_process_file"
                case "raw_processstop":
                    event_type = "raw_process_stop"
                case "raw_registry":
                    event_type = "raw_process_registry"
            local_file_path = (
                f"{local_path}/raw_sensor/{event_type}/dayPK={datadpk}/hourPK={datahpk}"
            )
        os.makedirs(local_file_path, exist_ok=True)
        shutil.copy2(srcdir + "/" + file, local_file_path + "/" + file)